In [1]:
import re
import os
from bisect import bisect_left
import time
import Classes

ImportError: No module named 'Classes'

In [9]:
# import the lexicon
one = time.time()
with open('tsikchen.txt', 'r', -1, 'utf-8-sig') as f:
    lexicon = [line.strip() for line in f.readlines()]
# add all the particles
lexicon.extend(['གི', 'ཀྱི', 'གྱི', 'ཡི', 'གིས', 'ཀྱིས', 'གྱིས', 'ཡིས', 'སུ', 'ཏུ', 'དུ', 'རུ', 'སྟེ', 'ཏེ', 'དེ', 'ཀྱང', 'ཡང', 'འང', 'གམ', 'ངམ', 'དམ', 'ནམ', 'བམ', 'མམ', 'འམ', 'རམ', 'ལམ', 'སམ', 'ཏམ', 'གོ', 'ངོ', 'དོ', 'ནོ', 'མོ', 'འོ', 'རོ', 'ལོ', 'སོ', 'ཏོ', 'ཅིང', 'ཅེས', 'ཅེའོ', 'ཅེ་ན', 'ཅིག', 'ཞིང', 'ཞེས', 'ཞེའོ', 'ཞེ་ན', 'ཞིག', 'ཤིང', 'ཤེའོ', 'ཤེ་ན', 'ཤིག', 'ལ', 'ན', 'ནས', 'ལས', 'ནི', 'དང', 'གང', 'ཅི', 'ཇི', 'གིན', 'གྱིན', 'ཀྱིན', 'ཡིན', 'པ', 'བ', 'པོ', 'བོ'])
# add all Monlam verbs
with open('monlam1_verbs.txt', 'r', -1, 'utf-8-sig') as f:
    monlam_verbs = [line.strip() for line in f.readlines()]
for entry in monlam_verbs:
    verb = entry.split(' | ')[0]
    if verb not in lexicon:
        lexicon.append(verb)
lexicon = sorted(lexicon)
len_lexicon = len(lexicon)

FileNotFoundError: [Errno 2] No such file or directory: 'tsikchen.txt'

In [6]:
# import the Monlam POS tags
with open('./monlam1_pos.txt', 'r', -1, 'utf-8-sig') as f: # Monlam
    monlam = [line.strip() for line in f.readlines()]
monlam_pos = {}
for line in monlam:
    parts = line.split('—') # Monlam
    monlam_pos[parts[0]] = parts[1] 

# import the Hill POS tags
with open('hill_pos.txt', 'r', -1, 'utf-8-sig') as f: # Hill
    hill = [line.strip() for line in f.readlines()]
hill_pos = {}
for line in hill:
    parts = line.split('—') # Hill
    hill_pos[parts[0]] = parts[1]
two = time.time()

def get_pos(dict, key):
    if key.endswith('་'):
        key = key[:-1]
    return dict.get(key)

def get_main_pos(key, num):
    if num == 0:
        Dict = monlam_pos
    if num == 1:
        Dict = hill_pos
    pos = get_pos(Dict, key)

    main_pos = []
    if pos != None:
        # Monlam
        if num == 0:
            parts = pos.split('/')
            for part in parts:
                if part != '':
                    main_pos.append(part.split(':')[0])

        # Hill
        if num == 1:
            parts = pos.split('/')
            for part in parts:
                if part != '':
                    if '.' in part:
                        main_pos.append(part.split('.')[0])
                    else:
                        main_pos.append(part)        
    return main_pos

In [3]:
def search(List, entry):
    global len_lexicon
    index = bisect_left(List, entry, 0, len_lexicon)
    return(True if index != len_lexicon and List[index] == entry else False)
    
def isWord(maybe):
    final = False
    if search(lexicon, maybe) == True:
        final = True
    elif search(lexicon, re.sub(merged_part, '', maybe)) == True:
        final = True
    else:
        last_syl = ''
        if '་' in maybe:
            last_syl = maybe.split('་')[-1]
            if SylComponents().get_info(last_syl) == 'thame':
                maybe = re.sub(merged_part, '', maybe) + 'འ'
        else:
            if SylComponents().get_info(maybe) == 'thame':
                maybe = re.sub(merged_part, '', maybe) + 'འ'
        if search(lexicon, maybe) == True:
            final = True
    return final

def process(list1, list2, num):
    global c
    word = '་'.join(list1[c:c+num])
    if search(lexicon, word) == False:
        maybe = re.split(merged_part, word)
        if search(lexicon, maybe[0]) == False and search(lexicon, maybe[0]+'འ'):
            list2.append(maybe[0]+'འ')
        else:
            list2.append(maybe[0])
        list2.append(maybe[1]+'་')
        #del list1[:num]
        c = c + num
    else:
        list2.append(word+"་")
        #del list1[:num]
        c = c + num

In [4]:
mark_unknown = 0
mark = '*'

In [ ]:
for file in os.listdir('./IN/'):
    if file.endswith(".txt"):
    #todo - replace \n by \s try: with open('drugs') as temp_file: \n drugs = [line.rstrip('\n') for line in temp_file]
        try:
            with open('./IN/' + file, 'r', -1, 'utf-8-sig') as f:
                current_file = f.read().replace('\n', '').replace('\r\n', '').replace('༌', '་')

        except:
            print("Save all IN files as UTF-8 and try again.")
            #input()
    else:
        print("\nSave all IN files as text files and try again.")
        #input()
    #start = time.time()
    ######################
    # Segmentation process

    merged_part = r'(ར|ས|འི|འམ|འང|འོ)$'
    punct_regex = r'([༄༅༆༇༈།༎༏༐༑༔\s]+)'
    
    paragraphs = re.split(punct_regex, current_file)
    if paragraphs[0] == '':
        del paragraphs[0]
    if paragraphs[len(paragraphs)-1] == '':
        del paragraphs[len(paragraphs)-1]
        
    text = []
    for par in paragraphs:
        words = []
        if re.match(punct_regex, par) == None:
            syls = par.split('་')
            if syls[0] == '':
                del syls[0]
            if syls[len(syls)-1] == '':
                del syls[len(syls)-1]
            #print('\t', syls)
            c = 0
            while c < len(syls):
                #if   isWord(syls[c:c+4]): process(syls, words, 4)
                if len(syls[c:c+3]) == 3 and isWord('་'.join(syls[c:c+3])): 
                    #print('3', syls[c:c+3])
                    process(syls, words, 3)
                elif len(syls[c:c+2]) == 2 and isWord('་'.join(syls[c:c+2])): 
                    #print('2', syls[c:c+2])
                    process(syls, words, 2)
                elif len(syls[c:c+1]) == 1 and isWord('་'.join(syls[c:c+1])): 
                    #print('1', syls[c:c+1])
                    process(syls, words, 1)
                else:
                    if mark_unknown == 0:
                        words.append('་'.join(syls[c:c+1])+'་')
                    elif mark_unknown == 1:
                        words.append(mark+'་'.join(syls[c:c+1])+'་')
                        print(syls[c:c+1])
                    c = c + 1
            paragraph = ' '.join(words)
            if not paragraph.endswith('ང་'):
                paragraph = paragraph[:-1]
            #########
            # add spaces at all tseks
            paragraph = re.sub(r'་([^ ])', r'་ \1', paragraph)
            #
            #########
            text.append(paragraph)
        else:
            text.append(par)
    #
    ######################
    text = ''.join(text)
    
    ######################
    # Transpose to AntTib
    text = AntTib().to_pw_text(text)
    #
    ######################
    
    # write output
    with open('./OUT/' + 'anttib_' + file, 'w', -1, 'utf-8-sig') as f:
        f.write(text)

In [1]:
print(AntTib().to_pw_text('མདུན'))